In [2]:
# função fit da FuzzyARTMAP:

# ```
def fit(
    self,
    X: np.ndarray,
    y: np.ndarray,
    max_iter: int = 1,
    match_tracking: Literal["MT+", "MT-", "MT0", "MT1", "MT~"] = "MT+",
    epsilon: float = 1e-10,
    verbose: bool = False,
    leave_progress_bar: bool = True,
):
    """Fit the model to the data.

    Parameters
    ----------
    X : np.ndarray
        Data set A.
    y : np.ndarray
        Data set B.
    max_iter : int, default=1
        Number of iterations to fit the model on the same data set.
    match_tracking : Literal, default="MT+"
        Method to reset the match.
    epsilon : float, default=1e-10
        Small value to adjust the vigilance.
    verbose : bool, default=False
        non functional. Left for compatibility
    leave_progress_bar : bool, default=True
        non functional. Left for compatibility

    Returns
    -------
    self : SimpleARTMAP
        The fitted model.

    """
    X_ = np.ascontiguousarray(X, dtype=np.float64)
    y_ = np.ascontiguousarray(y, dtype=np.int32)
    SimpleARTMAP.validate_data(self, X_, y_)
    self.classes_ = unique_labels(y_)
    self.labels_ = y_
    self.module_a.W = []
    self.module_a.labels_ = np.zeros((X_.shape[0],), dtype=int)

    la, W, cl = FitFuzzyARTMAP(
        X_,
        y_,
        rho=self.module_a.params["rho"],
        alpha=self.module_a.params["alpha"],
        beta=self.module_a.params["beta"],
        MT=match_tracking,
        epsilon=epsilon,
        weights=None,
        cluster_labels=None,
    )
    self._synchronize_cpp_results(la, W, cl)
    self.module_a.is_fitted_ = True
    return self
# ```

# função fit da SimpleARTMAP:

# ```
def fit(
    self,
    X: np.ndarray,
    y: np.ndarray,
    max_iter=1,
    match_tracking: Literal["MT+", "MT-", "MT0", "MT1", "MT~"] = "MT+",
    epsilon: float = 1e-10,
    verbose: bool = False,
    leave_progress_bar: bool = True,
):
    """Fit the model to the data.

    Parameters
    ----------
    X : np.ndarray
        Data set A.
    y : np.ndarray
        Data set B.
    max_iter : int, default=1
        Number of iterations to fit the model on the same data set.
    match_tracking : Literal, default="MT+"
        Method to reset the match.
    epsilon : float, default=1e-10
        Small value to adjust the vigilance.
    verbose : bool, default=False
        If True, displays a progress bar during training.
    leave_progress_bar : bool, default=True
        If True, leaves thge progress of the fitting process. Only used when
        verbose=True

    Returns
    -------
    self : SimpleARTMAP
        The fitted model.

    """
    # Check that X and y have correct shape
    SimpleARTMAP.validate_data(self, X, y)
    # Store the classes seen during fit
    self.classes_ = unique_labels(y)
    self.labels_ = y
    # init module A
    self.module_a.W = []
    self.module_a.labels_ = np.zeros((X.shape[0],), dtype=int)

    for _ in range(max_iter):
        if verbose:
            from tqdm import tqdm

            x_y_iter = tqdm(
                enumerate(zip(X, y)),
                total=int(X.shape[0]),
                leave=leave_progress_bar,
            )
        else:
            x_y_iter = enumerate(zip(X, y))
        for i, (x, c_b) in x_y_iter:
            self.module_a.pre_step_fit(X)
            c_a = self.step_fit(
                x,
                c_b,
                match_tracking=match_tracking,
                epsilon=epsilon,
            )
            self.module_a.labels_[i] = c_a
            self.module_a.post_step_fit(X)
    return self
# ```

SyntaxError: invalid syntax (3032739695.py, line 1)